In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from Fixed_Income_Toolbox import *

In [3]:
maturity = np.arange(1,7)
spot = [0.05, 0.055, 0.057, 0.059, 0.06, 0.061]
price = []
for i in range(len(maturity)):
    bond = ZeroCouponBond(100, maturity[i])
    price.append(bond.get_price(spot[i]))
price

[95.23809523809524,
 89.84524157139327,
 84.6788669093383,
 79.50897588580243,
 74.7258172866057,
 70.09833403416522]

In [32]:
def ho_and_lee_calibrate(level, spot, vol, price, fv=100):
    calibrate_lv = [s + vol for s in spot[level - 1]] + [spot[level - 1][-1] - vol] # construct the last level of spot
    def function(x, *args):
        calibrate_lv, spot, price, fv = args
        price_tree = [fv / (1 + spot + x) for spot in calibrate_lv]
        for i in reversed(range(len(price_tree))):
            for j in range(i):
                price_tree[j] = (0.5 * price_tree[j] + 0.5 * price_tree[j+1]) / (1 + spot[i][j])
        return price_tree[0] - price
    args = (calibrate_lv, spot, price, fv)
    m = fsolve(function, 0.01, args=args)
    return [s + m[0] for s in calibrate_lv]

In [33]:
# test calibration using the example in class
spot_dict = {1:[0.05263], 2:[0.07593, 0.03593]}
ho_and_lee_calibrate(3, spot_dict, 0.02, 85)

[0.09996345790671082, 0.05996345790671081, 0.01996345790671081]

In [34]:
# construct the ho and lee tree
vol = 0.015
spot_dict = {1:[spot[0]]}
for lv in range(2, len(price)+1):
    spot_dict[lv] = ho_and_lee_calibrate(lv, spot_dict, vol, price[lv-1])
spot_dict

{1: [0.05],
 2: [0.07523602642117286, 0.04523602642117285],
 3: [0.09164759479639102, 0.061647594796391016, 0.03164759479639102],
 4: [0.11129248303472124,
  0.08129248303472124,
  0.05129248303472124,
  0.021292483034721237],
 5: [0.1261246338504627,
  0.09612463385046267,
  0.06612463385046267,
  0.03612463385046268,
  0.006124633850462678],
 6: [0.14418404774940974,
  0.11418404774940971,
  0.08418404774940971,
  0.05418404774940973,
  0.024184047749409722,
  -0.005815952250590278]}